In [ ]:
import piexif
from pathlib import Path

In [73]:
import pandas as pd
from xml.dom.minidom import Document
import os
import cv2
from pathlib import Path
import glob
from tqdm import tqdm

In [22]:
def bytes2hex(bytes):
    print('关键码转码……');
    num = len(bytes)
    hexstr = u""
    for i in range(num):
        t = u"%x" % bytes[i]
        if len(t) % 2:
            hexstr += u"0"
        hexstr += t
    return hexstr.upper()

In [45]:
with open('test.jpg', 'rb') as f:
    img_bin20 = f.read(16)

In [38]:
def is_jpg(filename):
    data = open(filename,'rb').read(11)
    if data[:4] != b'\xff\xd8\xff\xe0' and data[:4]!=b'\xff\xd8\xff\xe1': 
        return False
    if data[6:] != b'JFIF\0' and data[6:] != b'Exif\0': 
        return False
    return True

In [71]:
!ls /nas/chenyi/datasets_cls/gallery_detect/gallery_csv

gall_df.csv  gallery00.csv  test.csv


In [74]:
csv_path = '/nas/chenyi/datasets_cls/gallery_detect/gallery_csv/gall_df.csv'
gall_df = pd.read_csv(csv_path)

In [76]:
gall_df.head(1)

,oss_sp,stem
0,/nas/oss/gallery/69/31/69313640-af52-465e-88c8...,69313640-af52-465e-88c8-c0a0099ca5c7-699544


In [75]:
len(gall_df)

1661397

In [ ]:
myPyModule99/utils/data

In [79]:
import sys,os
sys.path.append('../../')
from utils.data.transfer import run_task_multithread

In [80]:
def get_jpeg(sp):
    return is_jpg(sp)

In [82]:
%%time
flag_list = []
for p in tqdm(gall_df['oss_sp'].values):
    flag_list.append(get_jpeg(p))
    # break

100%|████████████████████████████████████████████████████████████| 1661397/1661397 [2:54:01<00:00, 159.12it/s]

CPU times: user 2min 7s, sys: 2min 43s, total: 4min 51s
Wall time: 2h 54min 1s


In [83]:
gall_df['jpeg'] = flag_list

In [84]:
gall_df.to_csv('gall_df.csv', index=False)

In [85]:
gall_df.value_counts('jpeg')

jpeg
True     1477501
False     183896
dtype: int64

In [88]:
format_clr_df = gall_df[~gall_df['jpeg']].copy(deep=True)
len(format_clr_df)

183896

In [96]:
format_clr_df.head(1)

,oss_sp,stem,jpeg,oss_tp
7,/nas/oss/gallery/69/31/69313024-7305-4862-b853...,69313024-7305-4862-b853-96f2029d32df-303224,False,/nas/chenyi/datasets_cls/gallery_detect/gall_n...


In [97]:
format_clr_df.head(1).values

array([['/nas/oss/gallery/69/31/69313024-7305-4862-b853-96f2029d32df-303224.jpg',
        '69313024-7305-4862-b853-96f2029d32df-303224', False,
        PosixPath('/nas/chenyi/datasets_cls/gallery_detect/gall_nojpeg/69/31/69313024-7305-4862-b853-96f2029d32df-303224.jpg')]],
      dtype=object)

In [124]:
root_save = Path('/nas/chenyi/datasets_cls/gallery_detect/gall_nojpeg')
root_chg_jpeg = Path('/nas/chenyi/datasets_cls/gallery_detect/chg2jpeg')

In [95]:
format_clr_df['oss_tp'] = format_clr_df['oss_sp'].apply(lambda x: root_save /Path(x).parts[-3]/Path(x).parts[-2]/Path(x).name)

In [125]:
format_clr_df['jpeg_tp'] = format_clr_df['oss_tp'].apply(lambda x: root_chg_jpeg /Path(x).parts[-3]/Path(x).parts[-2]/Path(x).name)

In [146]:
df_chg = format_clr_df[int(8e4):]

In [147]:
import shutil
sp_noexist_list = []
for p in tqdm(df_chg['oss_sp'].values):
    sp = Path(p)
    tp = root_save /sp.parts[-3] /sp.parts[-2] /sp.name
    if not sp.is_file():
        sp_noexist_list.append(sp)
        continue
    if not tp.parent.is_dir():
        tp.parent.mkdir(parents=True, exist_ok=True)
    if tp.is_file():
        continue
    shutil.copyfile(sp, tp)
    # break

100%|█████████████████████████████████████████████████████████████████| 103896/103896 [36:18<00:00, 47.68it/s]


In [148]:
def copy_error_file(sp):
    tp = root_save /sp.parts[-3] /sp.parts[-2] /sp.name
    if not tp.parent.is_dir():
        tp.parent.mkdir(parents=True, exist_ok=True)
    if not tp.is_file():
        shutil.copyfile(sp, tp)

##### change 2 jpeg

In [ ]:
except_list = []
df_chg = format_clr_df
for p in tqdm(df_chg['oss_tp'].values):
    sp = Path(p)
    tp = root_chg_jpeg /sp.parts[-3] /sp.parts[-2] /sp.name
    if not sp.is_file():
        continue
    if not tp.parent.is_dir():
        tp.parent.mkdir(parents=True, exist_ok=True)
    if tp.is_file():
        continue
    img_cv2 = cv2.imread(str(sp))
    try:
        cv2.imwrite(str(tp), img_cv2)
    except:
        except_list.append(sp)
    # break

 71%|███████████████████████████████████████████▍                 | 130861/183896 [3:37:47<1:03:04, 14.02it/s]

In [ ]:
len(except_list)